# Multiword expressions identification and extraction

In [1]:
import spacy
import pandas as pd
import re
import os

from random import sample
from collections import Counter


In [2]:
path_to_files = "ustawy"

def read_normalized_documents(): 
    file_names = os.listdir(path_to_files)
    return {
        name: _normalize_document(_read_document(name, path_to_files))
        for name in file_names
        if name.endswith(".txt")
    }

def _read_document(name: str, path: str):
    with open(os.path.join(path, name), 'r') as f:
        return f.read()


def _normalize_document(document: str):
    return re.sub(r"\s+", " ", document).lower()

In [3]:
# Create a pipe that converts lemmas to lower case:
def lower_case_lemmas(doc) :
    for token in doc :
        token.lemma_ = token.lemma_.lower()
    return doc

In [4]:
nlp = spacy.load("pl_core_news_sm")  # had to be downloaded separately with "python3 -m spacy download pl_core_news_sm"
tokenizer = nlp.tokenizer
documents = read_normalized_documents()

In [9]:
tokenized_documents = {name: [t.text for t in tokenizer(text)] for name, text in documents.items()}
print(type(tokenized_documents))
for name, tokens in sample(list(tokenized_documents.items()), 10):
    print(f"{name}: {sample(tokens, 10)}", type(tokens))

<class 'dict'>
2003_1068.txt: [',', 'o', 'mowa', 'mowa', '1', 'ustawy', 'górnictwa', '.', 'nr', 'poz'] <class 'list'>
2001_1645.txt: ['polskiej', 'przywilejach', 'ratyfikacji', 'jądrowych', '2004', 'i', 'protokołu', 'przywilejach', 'o', '.'] <class 'list'>
1996_347.txt: ['.', 'z', '.', '.', 'z', 'środków', '1', 'prezydent', 'dnia', '.'] <class 'list'>
1994_601.txt: ['art', '1998', 'poz', '.', '1998', 'dnia', '.', 'lipca', 'nr', 'art'] <class 'list'>
2003_1759.txt: ['.', 'osobie', 'nr', ')', 'postoju', 'kontroli', 'ruchu', 'na', ')', 'i'] <class 'list'>
1997_348.txt: ['których', 'karę', 'nr', 'z', 'organizacje', 'nie', 'i', 'energetycznych', 'w', 'rozporządzenia'] <class 'list'>
2004_76.txt: ['.', 'określone', ':', 'zaplanowano', 'brzmienie', ',', '873', ')', 'do', 'lipca'] <class 'list'>
1997_805.txt: ['bez', '198', 'w', 'posiadają', '11', 'cmentarzu', 'nagrobków', 'ustawa', 'miejsca', 'ogłoszenia'] <class 'list'>
2001_1086.txt: ['wykonywania', 'sprzedaż', ')', 'umowy', 'według', 's.a'

Lowercase:

2. Compute bigram counts of downcased tokens

In [26]:
bigrams = [
    (f, s)
    for tokens in tokenized_documents.values()
    for (f, s) in zip(tokens[:-1], tokens[1:])
]
print(type(bigrams),bigrams[0:100])

<class 'list'> [(' ', 'dz.u'), ('dz.u', '.'), ('.', 'z'), ('z', '2001'), ('2001', 'r'), ('r', '.'), ('.', 'nr'), ('nr', '81'), ('81', ','), (',', 'poz'), ('poz', '.'), ('.', '874'), ('874', 'ustawa'), ('ustawa', 'z'), ('z', 'dnia'), ('dnia', '21'), ('21', 'czerwca'), ('czerwca', '2001'), ('2001', 'r'), ('r', '.'), ('.', 'o'), ('o', 'zmianie'), ('zmianie', 'ustawy'), ('ustawy', 'o'), ('o', 'państwowej'), ('państwowej', 'straży'), ('straży', 'pożarnej'), ('pożarnej', 'art'), ('art', '.'), ('.', '1'), ('1', '.'), ('.', 'w'), ('w', 'ustawie'), ('ustawie', 'z'), ('z', 'dnia'), ('dnia', '24'), ('24', 'sierpnia'), ('sierpnia', '1991'), ('1991', 'r'), ('r', '.'), ('.', 'o'), ('o', 'państwowej'), ('państwowej', 'straży'), ('straży', 'pożarnej'), ('pożarnej', '('), ('(', 'dz.u'), ('dz.u', '.'), ('.', 'nr'), ('nr', '88'), ('88', ','), (',', 'poz'), ('poz', '.'), ('.', '400'), ('400', 'z'), ('z', '1992'), ('1992', 'r'), ('r', '.'), ('.', 'nr'), ('nr', '21'), ('21', ','), (',', 'poz'), ('poz', '.')

In [34]:
# Calculate bigrams count

freq_list = Counter(bigrams)

print(sample(list(freq_list.items()), 15))

[(('obowiązku', 'zapobiegania'), 1), (('sposób', 'stronniczy'), 2), (('wymagają', 'wyłączenia'), 1), (('518', 'otrzymuje'), 1), (('kontrakty', 'z'), 1), (('każdym', 'województwie'), 1), (('uwagę', 'stanowiska'), 3), (('o', 'wprowadzonych'), 4), (('na', 'kwasach'), 1), (('przepisów', 'działalność'), 1), (('obcym', 'nowożytnym'), 1), (('tym', 'kary'), 1), (('wymagające', 'oznakowania'), 1), (('obliczając', 'należne'), 2), (('składa', 'prokuratorowi'), 1)]


PMI:
```
log(p(a,b) / ( p(a) * p(b) ))
```
programmatically:
```
def pmi(word1, word2, unigram_freq, bigram_freq):
  prob_word1 = unigram_freq[word1] / float(sum(unigram_freq.values()))
  prob_word2 = unigram_freq[word2] / float(sum(unigram_freq.values()))
  prob_word1_word2 = bigram_freq[" ".join([word1, word2])] / float(sum(bigram_freq.values()))
  return math.log(prob_word1_word2/float(prob_word1*prob_word2),2) 

```